## <center> *Модель прогнозирования стоимости жилья для агентства недвижимости*

# <center> **Часть I. Обработка данных.**

Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

import ast

from functions import *

import warnings
warnings.filterwarnings('ignore')

## Данные

Читаем данные

In [2]:
data = pd.read_csv('data/data.csv')

display(data.head())
print(f'{data.shape[0]} rows, {data.shape[1]} columns')

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


377185 rows, 18 columns


Основная информация о данных

In [3]:
get_data_info(data)

,Nulls,Uniques,Dtypes
status,10.58,160,object
private pool,98.89,2,object
propertyType,9.21,1281,object
street,0.00,337077,object
baths,28.19,230,object
homeFacts,0.00,321009,object
fireplace,72.66,1653,object
city,0.01,2027,object
schools,0.00,297365,object
sqft,10.76,25406,object


In [4]:
data['MlsId'] = data['MlsId'].fillna('') + data['mls-id'].fillna('')
data['MlsId'] = data['MlsId'].apply(lambda x: np.nan if x == '' else x)

t_col = data.shape[0] * 0.7
data.dropna(axis=1, thresh=t_col, inplace=True)

t_row = data.shape[1] * 0.7
data.dropna(axis=0, thresh=t_row, inplace=True)

data.reset_index(drop=True, inplace=True)

get_data_info(data)

,Nulls,Uniques,Dtypes
status,10.18,159,object
propertyType,5.13,1281,object
street,0.00,321390,object
baths,24.81,230,object
homeFacts,0.00,317304,object
city,0.01,2000,object
schools,0.00,286152,object
sqft,6.62,25397,object
zipcode,0.00,4499,object
beds,21.57,193,object


## Обработка признаков

In [5]:
better_data = pd.DataFrame(index=data.index)

better_data

""
0
1
2
3
4
...
360187
360188
360189
360190


### Статус продажи

In [6]:
def fix_feature(feature, 
                filling='Unknown',
                lower=True):
    
    x = feature.fillna(str(filling))
    
    if lower:
        x = x.apply(lambda x: x.lower())
    
    return x

for col in data.columns:
    data[col] = fix_feature(data[col])

In [7]:
get_populars(data['status'])

['for sale',
 'active',
 'unknown',
 'foreclosure',
 'new construction',
 'pending',
 'pre-foreclosure',
 'p',
 'pre-foreclosure / auction',
 'under contract show',
 ' / auction',
 'under contract   showing',
 'active under contract',
 'under contract',
 'new',
 'contingent',
 'price change',
 'auction',
 'a active',
 'for rent',
 'foreclosed',
 'under contract backups',
 'contingent finance and inspection',
 'recently sold',
 'pending continue to show',
 'option pending',
 'back on market',
 'contingent show',
 'pending taking backups',
 'option contract',
 'active with offer',
 'pending with contingencies',
 'active backup',
 'c',
 'contract p',
 'pi',
 'p pending sale',
 'listing extended',
 'auction - active',
 'due diligence period',
 'contract contingent on buyer sale',
 'c continue show',
 'pending - taking backups',
 'offer pending signature',
 'active/contingent',
 'pending inspection',
 'contingent take backup',
 'contingent   show',
 'pending in',
 'active with contingencies

In [8]:
data['status'] = (data['status']
                  .apply(lambda x: 'pending' if ((x == 'p') or 
                                                 (len(x) == 2 and 'p' in x)) else x))


def get_dummy_features(f_name, 
                       rep_dict=None, 
                       labels=None, 
                       data=data, 
                       new_data=better_data,
                       get_replaced=False):
    feature = data[f_name]
    
    if rep_dict is not None:
        for key, val in rep_dict.items():
            feature = feature.apply(lambda x: x.replace(key, val))
        
    if get_replaced:
        return feature
    
    if labels is not None:
        for label in labels:
            new_data[f_name+' '+label] = (data[f_name]
                                        .apply(lambda x: 1 if label in x else 0)
                                        .astype('uint8'))
            
        named_cols = list(filter(lambda x: f_name in x, new_data.columns))
        
        new_data[f_name+' other'] = (new_data.loc[:, named_cols]
                                    .sum(axis=1)
                                    .apply(lambda x: 1 if x == 0 else 0)
                                    .astype('uint8'))
    
    return new_data 


status_dict = {'forclosed': 'forclosure', 
               'pre forclosure': 'pre-forclosure', 
               'c': 'contingency', 
               'contingency': 'contingencies', 
               'contingencies': 'contingent'}

statuses = ['for sale', 'coming soon', 'new', 
            'foreclosure', 'pre-foreclosure', 'pending', 
            'auction', 'active', 'show',  
            'for rent', 'sold', 'under contract', 
            'contingent', 'change', 'unknown']

better_data = get_dummy_features('status', status_dict, statuses)


get_sums(better_data, 'status')

status for sale           199477
status coming soon           110
status new                  6153
status foreclosure         10109
status pre-foreclosure      3529
status pending              6878
status auction              2954
status active              93439
status show                 2329
status for rent              412
status sold                  240
status under contract       3661
status contingent           1023
status change                558
status unknown             36678
status other                 722
dtype: int64

### Тип недвижимости

In [9]:
get_populars(data['propertyType'])

['single-family home',
 'single family',
 'condo',
 'single family home',
 'lot/land',
 'unknown',
 'townhouse',
 'land',
 'multi-family',
 'condo/townhome/row home/co-op',
 'traditional',
 'coop',
 'multi family',
 'high rise',
 'ranch',
 'mobile/manufactured',
 'detached, one story',
 'single detached, traditional',
 'contemporary',
 'multi-family home',
 '1 story',
 'colonial',
 'mobile / manufactured',
 'contemporary/modern',
 '2 stories',
 'apartment',
 'mfd/mobile home',
 'single detached',
 'detached, two story',
 'one story',
 'transitional',
 'cooperative',
 'florida',
 'bungalow',
 'farms/ranches',
 'two story',
 'garden home',
 '2 story',
 'cape cod',
 '2 stories, traditional',
 'spanish/mediterranean',
 'other style',
 '1 story, traditional',
 'other',
 'condo/townhome, traditional',
 'craftsman',
 'contemporary/modern, traditional',
 'condo/townhome',
 'singlefamilyresidence',
 'condo/townhome, contemporary/modern',
 'single detached, contemporary/modern',
 'multiple occup

In [10]:
propdict = {'single family': 'tiny',
            'tiny': 'detached', 
            'detached': 'single-family', 
            'co-op': 'coop', 
            'townhome': 'townhouse', 
            'manufactured': 'prefab', 
            'prefab': 'mo2 le',
            'mo2le': 'mobile'} 

proptypes = ['single-family', 'condo', 
             'land', 'townhouse', 'multi', 
             'coop', 'traditional', 'ranch', 
             'mobile', 'detached', 'contemporary', 
             'modern', 'apartment', 'story', 
             'stories', 'high', 'colonial']

better_data = get_dummy_features('propertyType', propdict, proptypes)


better_data['propertyType unknown'] = (data['propertyType']
                                       .apply(lambda x: 1 if x in ('unknown', ' ') else 0)
                                       .astype('uint8'))

get_sums(better_data, 'propertyType')

propertyType single-family     92033
propertyType condo             51422
propertyType land              31458
propertyType townhouse         18572
propertyType multi             12485
propertyType coop               3751
propertyType traditional        9679
propertyType ranch              3193
propertyType mobile             3491
propertyType detached           5822
propertyType contemporary       4436
propertyType modern             2336
propertyType apartment           910
propertyType story              5913
propertyType stories            1745
propertyType high               2041
propertyType colonial           1401
propertyType other            119337
propertyType unknown           18617
dtype: int64

### Количество ванных

In [11]:
get_populars(data['baths'])

['unknown',
 '2 baths',
 '3 baths',
 '2',
 '2.0',
 '4 baths',
 '3.0',
 '3',
 'bathrooms: 2',
 '2.5',
 'bathrooms: 3',
 '1',
 '1.0',
 '5 baths',
 '4.0',
 '2.5 baths',
 '0',
 '4',
 '3.5',
 'bathrooms: 1',
 '2 ba',
 '6 baths',
 'bathrooms: 4',
 '1,500',
 '3 ba',
 '1.5',
 '3.5 baths',
 '2,000',
 '1,750',
 '3,000',
 '5.0',
 '2,250',
 '1,000',
 '5',
 '7 baths',
 '1,250',
 '4.5',
 '2,750',
 '2,500',
 'bathrooms: 5',
 '~',
 '1 ba',
 '6.0',
 '4 ba',
 '3,500',
 '8 baths',
 '-- baths',
 '6',
 '4.5 baths',
 '7.0',
 'bathrooms: 6',
 '9 baths',
 '5.5',
 '7',
 '4,000',
 '5 ba',
 '2.5+',
 '1.5 baths',
 '750',
 '5,000',
 '8.0',
 '3.5+',
 'sq. ft. ',
 '10 baths',
 '8',
 '4.5+',
 'bathrooms: 7',
 '5.5+',
 '6.5',
 '9.0',
 '6 ba',
 '11 baths',
 '1.75 baths',
 'bathrooms: 8',
 '5.5 baths',
 '12 baths',
 '9',
 '0 / 0',
 '6.5+',
 '7 ba',
 '1.5+',
 '10.0',
 '10',
 '2.1 baths',
 '2.5 ba',
 '13 baths',
 '11',
 '8 ba',
 '2.75 baths',
 'bathrooms: 9',
 '16 baths',
 '7.5',
 '7.5+',
 '11.0',
 '14 baths',
 'bathrooms

In [12]:
def get_baths(x):    
    if '.' in x:
        x = x.split('.')[0]
    
    if '/' in x:
        x = x.split(' / ')[0]
        
        if '-' in x:
            x = x.split('-')[0]

    if (not x.isnumeric()) or (',' in x):
        return -1
    else:
        return x


bath_dict = {' baths': ' ba',
             ' ba': '',  
             'bathrooms: ': '', 
             '.0': '', 
             '+': ''}

better_data['baths'] = (get_dummy_features('baths', 
                                           bath_dict, 
                                           get_replaced=True)
                        .apply(get_baths)
                        .astype(int))


get_vc(better_data['baths'])

baths
 2      114724
-1      105240
 3       70534
 4       27609
 1       18780
 5        9747
 6        4497
 0        3918
 7        2143
 8        1190
 9         552
 10        342
 750       237
 11        190
 12        138
 13         74
 16         52
 14         46
 18         32
 15         28
Name: count, dtype: int64

### Количество спален

In [13]:
get_populars(data['beds'])

['unknown',
 '3 beds',
 '4 beds',
 '3',
 '2 beds',
 '4',
 '2',
 'baths',
 '3 bd',
 '5 beds',
 '4 bd',
 '3.0',
 '5',
 '4.0',
 '2 bd',
 '1',
 '6 beds',
 '5 bd',
 '2.0',
 '6',
 '5.0',
 '0',
 '7 beds',
 '1 bd',
 '-- bd',
 '8 beds',
 'bath',
 '6 bd',
 '7',
 '1.0',
 '9 beds',
 ' ',
 '8',
 '6.0',
 '0.0',
 '10 beds',
 '7 bd',
 '12 beds',
 '8 bd',
 '9',
 '11 beds',
 '7.0',
 '10',
 '64 beds',
 '16 beds',
 '8.0',
 '11',
 '9 bd',
 '12',
 '13 beds',
 '14 beds',
 '18 beds',
 '10 bd',
 '15 beds',
 '16',
 '12 bd',
 '24 beds',
 '14',
 '13',
 '20 beds',
 '10.0',
 '24',
 '11 bd',
 '15',
 '20',
 '9.0',
 '17 beds',
 '18',
 '32 beds',
 '28 beds',
 '16 bd',
 '22 beds',
 '26 beds',
 '24 bd',
 '13 bd',
 '34 beds',
 '21 beds',
 '19 beds',
 '14 bd',
 '17',
 '28',
 '8,276 sqft',
 '36 beds',
 '11.0',
 '22',
 '25 beds',
 '23 beds',
 '32',
 '5,227 sqft',
 '16.0',
 '42 beds',
 '18 bd',
 '15.0',
 '29',
 '12.0',
 '44 beds',
 '47 beds',
 '40 beds',
 '34',
 '40']

In [14]:
def get_beds(x):
    xsplit = x.split()
    
    exepts = ['unknown', 'sqft', 'acre', '-', '#']
    
    for ex in exepts:
        if ex in x:
            return -1
    
    if (('#' in xsplit) or
        (len(xsplit) == 0) or 
        ((len(xsplit) == 1) and ('bath' in xsplit[0]))):
        return -1
                
    for i in range(len(xsplit)):
        if (xsplit[i] == 'or') or ('bedrooms' in xsplit[i]):
            return xsplit[i-1]
            
    if not xsplit[0].isnumeric():
        return -1
    else:    
        return x


beds_dict = {' beds': ' bd', 
             ' bd': '', 
             '+': '', 
             '.0': ''}

better_data['beds'] = (get_dummy_features('beds', 
                                          beds_dict, 
                                          get_replaced=True)
                       .apply(get_beds)
                       .astype(int))


get_vc(better_data['beds'])

beds
 3     105780
-1      94986
 4      68919
 2      50125
 5      21699
 6       6580
 1       6118
 7       1825
 0       1430
 8       1217
 9        486
 10       274
 12       186
 11       134
 16        70
 13        48
 64        44
 14        43
 15        33
 18        31
Name: count, dtype: int64

### Площадь недвижимости

In [15]:
data['sqft'].value_counts(dropna=False)[-100:].index.tolist()

['total interior livable area: 4,881 sqft',
 '1,916,640 sqft',
 'total interior livable area: 3,585 sqft',
 'total interior livable area: 5,012 sqft',
 '12,426 sqft',
 '60,692 sqft',
 '6,871 sqft',
 'total interior livable area: 5,793 sqft',
 '5029',
 '7,689 sqft',
 '155,155 sqft',
 '6,072 sqft',
 '5568',
 '9,093',
 '7,417',
 '13,461 sqft',
 '52,708',
 '17,181 sqft',
 '4782',
 '9,807 sqft',
 '5,472 sqft',
 'total interior livable area: 6,784 sqft',
 'total interior livable area: 6,842 sqft',
 '163,080 sqft',
 '6650',
 '31,927 sqft',
 '8262',
 '6,009 sqft',
 '7,471',
 '6435',
 '10,544 sqft',
 '5093',
 '7,750 sqft',
 '7,254',
 '6,792 sqft',
 '47,272 sqft',
 '107,988 sqft',
 '10,268 sqft',
 '6,112 sqft',
 '256,750 sqft',
 'total interior livable area: 3,900 sqft',
 'total interior livable area: 7,328 sqft',
 'total interior livable area: 3,551 sqft',
 '12,031 sqft',
 '7240',
 'total interior livable area: 5,409 sqft',
 '9,722',
 '5,938',
 '5724',
 '7,131',
 '4433',
 '21,360',
 '11,392',
 

In [16]:
def get_sqft_area(x):
    
    xsplit = x.replace(',', '').split()
    q = xsplit[0]
    
    if q == 'total':
        return xsplit[-2]
    
    if not q.isnumeric():
        return '-1'
    else:
        return q
    
better_data['sqft area'] = (data['sqft']
                            .apply(get_sqft_area)
                            .astype(float)
                            .apply(lambda x: x / 1000 if x > 0 else x))


get_vc(better_data['sqft area'])

sqft area
-1.000    24653
 0.000    11853
 1.200     1424
 1.000     1026
 1.500     1014
 1.800      987
 1.100      937
 1.400      918
 2.000      871
 1.600      834
 0.800      754
 1.300      738
 1.440      736
 0.960      712
 0.900      702
 2.400      680
 2.200      610
 1.344      604
 1.152      601
 1.700      597
Name: count, dtype: int64

### Факты о недвижимости

In [17]:
data['homeFacts'] 

0         {'ataglancefacts': [{'factvalue': '2019', 'fac...
1         {'ataglancefacts': [{'factvalue': '2019', 'fac...
2         {'ataglancefacts': [{'factvalue': '1961', 'fac...
3         {'ataglancefacts': [{'factvalue': '2006', 'fac...
4         {'ataglancefacts': [{'factvalue': '', 'factlab...
                                ...                        
360187    {'ataglancefacts': [{'factvalue': '2010', 'fac...
360188    {'ataglancefacts': [{'factvalue': '1990', 'fac...
360189    {'ataglancefacts': [{'factvalue': '1924', 'fac...
360190    {'ataglancefacts': [{'factvalue': '1950', 'fac...
360191    {'ataglancefacts': [{'factvalue': '2019', 'fac...
Name: homeFacts, Length: 360192, dtype: object

In [18]:
homefacts = (data['homeFacts'].apply(lambda x: x
                                     .replace("'', ", "'no data', ")
                                     .replace(": none", ": 'no data'")))


hf_list_raw = []

for fact in homefacts:
    hf_le = ast.literal_eval(fact)
    hf_list_raw.append(hf_le['ataglancefacts'])


hf_list = []

for flist in hf_list_raw:
    hf_dict = {}
    
    for fdict in flist:
        v_list = list(fdict.values())
        
        hf_dict[v_list[1]] = v_list[0]
    
    hf_list.append(hf_dict)
    
    
hf_df = pd.DataFrame(hf_list)

for col in hf_df.columns:
    hf_df[col] = (fix_feature(hf_df[col], filling='None')
                  .apply(lambda x: np.nan if x == 'no data' else x))


display(hf_df)    
display(get_data_info(hf_df))

,year built,remodeled year,heating,cooling,parking,lotsize,price/sqft
0,2019,NaN,"central a/c, heat pump",NaN,NaN,NaN,$144
1,2019,NaN,NaN,NaN,NaN,5828 sqft,$159/sqft
2,1961,1967,forced air,central,attached garage,"8,626 sqft",$965/sqft
3,2006,2006,forced air,central,detached garage,"8,220 sqft",$371/sqft
4,NaN,NaN,NaN,NaN,NaN,"10,019 sqft",NaN
...,...,...,...,...,...,...,...
360187,2010,NaN,forced air,central,1 space,NaN,$564
360188,1990,1990,other,central,2 spaces,"8,500 sqft",$311
360189,1924,NaN,radiant,NaN,none,NaN,$337/sqft
360190,1950,1950,other,NaN,2,"1,600 sqft",$458/sqft


,Nulls,Uniques,Dtypes
year built,13.45,229,object
remodeled year,58.47,152,object
heating,28.16,1914,object
cooling,32.78,1433,object
parking,47.74,3319,object
lotsize,17.62,36184,object
price/sqft,13.54,6500,object


### *Возраст недвижимости*

In [19]:
hf_df['year built'] = (hf_df['year built']
                       .fillna('-1')
                       .astype(float))

better_data['property age'] = ((2023 - hf_df['year built'])
                               .apply(lambda x: -1 if x == 2024 else x)
                               .astype(int))

get_vc(better_data['property age'])

property age
-1      48455
 4      32081
 17      7984
 18      7480
 16      7114
 5       6832
 19      5484
 6       5167
 7       5086
 15      5001
 20      4549
 73      4504
 103     4279
 21      4252
 22      4194
 98      4153
 8       3860
 68      3660
 63      3629
 23      3603
Name: count, dtype: int64

### *Год реконструкции*

In [20]:
hf_df['remodeled year'] = (hf_df['remodeled year']
                           .fillna('2023')
                           .astype(int))

better_data['property remodeled'] = 2023 - hf_df['remodeled year']
  
get_vc(better_data['property remodeled'])

property remodeled
0     210605
17      5506
18      4768
16      4333
15      3763
19      3344
43      3310
53      3118
23      3054
20      2800
38      2794
21      2701
48      2669
33      2646
58      2597
40      2440
22      2434
28      2371
35      2332
61      2214
Name: count, dtype: int64

### *Отопление*

In [21]:
get_populars(hf_df['heating'])

['forced air',
 nan,
 'other',
 'electric',
 'gas',
 'heat pump',
 'central air',
 'central electric',
 'central',
 'central, electric',
 'baseboard',
 'wall',
 'electric heat',
 'heating system',
 'forced air, heat pump',
 'radiant',
 'central air, ceiling fan(s)',
 'natural gas heat',
 'central furnace',
 'forced air, gas',
 'central electric, zoned',
 'forced air heating',
 ', gas hot air/furnace',
 'gas heat',
 'forced air, natural gas',
 'central heating',
 'central gas',
 'central electric, heat pump',
 ', heat pump - heat',
 'central air conditioning',
 'electric, gas',
 'forced air, other',
 'central, gas',
 'refrigeration',
 'central, electric, heat pump',
 'refrigeration, ceiling fan(s)',
 'oil',
 'central, heat pump',
 'forced air, stove',
 'central air, central heat',
 'natural gas',
 'heat pump(s)',
 ', gas hot air/furnace, multizone heat',
 'central, gas, two or more units',
 'electric, heat pump',
 'baseboard, forced air',
 'central, electric, two or more units',
 'force

In [22]:
heatings = ['forced', 'air', 'none', 
            'electric', 'gas', 'heat pump', 
            'central', 'baseboard', 'wall', 
            'system', 'radiant', 'stove', 
            'zoned', 'refrigeration', 'oil']

hf_df['heating'].fillna('none', inplace=True)

better_data = get_dummy_features('heating', labels=heatings, data=hf_df)

get_sums(better_data, 'heating')

heating forced           140903
heating air              152932
heating none             101504
heating electric          29190
heating gas               18283
heating heat pump         13424
heating central           34542
heating baseboard          4611
heating wall               4510
heating system             2776
heating radiant            1916
heating stove               552
heating zoned              1557
heating refrigeration       586
heating oil                 363
heating other             31204
dtype: int64

### *Охлаждение*

In [23]:
get_populars(hf_df['cooling'])

['central',
 nan,
 'central air',
 'has cooling',
 'none',
 'central electric',
 'wall',
 'central gas',
 'central heating',
 'cooling system',
 'central a/c',
 'other',
 'central a/c (electric), central heat (gas)',
 'central a/c (electric), central heat (electric)',
 'refrigeration',
 'central, electric',
 'evaporative',
 'electric',
 'central air, gas hot air/furnace',
 'refrigeration, ceiling fan(s)',
 'central gas, zoned',
 'central, wall',
 'air conditioning-central',
 '2 or more units, central, electric',
 'has heating',
 'central air, gas (hot air)',
 'central electric, zoned',
 'central air, zoned',
 'solar, refrigeration',
 'central, other',
 'ceiling fan, central electric',
 'central cooling',
 'electric heating',
 'heat pump',
 'gas heating',
 'central a/c (electric), central heat (gas), zoned',
 'central electric, heat pump',
 'gas heating, forced air heating',
 'heat pump - ac',
 'central air, gas hot air/furnace, multizone a/c',
 'central electric, central gas',
 'forced

In [24]:
coolings = heatings + ['has', 'a/c', 'evaporative', 
                       'fan', 'conditioning', 'solar']

hf_df['cooling'].fillna('none', inplace=True)

better_data = get_dummy_features('cooling', labels=coolings, data=hf_df)

get_sums(better_data, 'cooling')

cooling forced              806
cooling air               20840
cooling none             125496
cooling electric          15514
cooling gas               10413
cooling heat pump          2670
cooling central          204383
cooling baseboard           292
cooling wall               5941
cooling system             2794
cooling radiant             166
cooling stove                51
cooling zoned              2523
cooling refrigeration      2764
cooling oil                  16
cooling has               10081
cooling a/c                8274
cooling evaporative        1317
cooling fan                2673
cooling conditioning       1482
cooling solar               691
cooling other              3583
dtype: int64

### *Парковка*

In [25]:
get_populars(hf_df['parking'])

[nan,
 'attached garage',
 '2 spaces',
 '1 space',
 'detached garage',
 'carport',
 'off street',
 '3 spaces',
 'carport, attached garage',
 '1',
 '4 spaces',
 '2',
 'none',
 'on street',
 'attached garage, detached garage',
 '0',
 'attached garage, carport',
 'parking desc',
 '6 spaces',
 'detached garage, attached garage',
 'driveway',
 '5 spaces',
 '4',
 'off street parking',
 'off street, attached garage',
 'parking type',
 '3',
 'carport, detached garage',
 'attached garage, garage - 2 car',
 'garage type',
 'parking yn',
 'off street, on street',
 'driveway, garage door opener',
 'garage - 2 car',
 'garage door opener',
 '888',
 'attached garage, off street',
 'off street, detached garage',
 'attached - front',
 'slab parking spaces',
 'detached garage, carport',
 '8 spaces',
 '6',
 'garage-attached',
 'electric door opener',
 'attached - side',
 'attached',
 'on street, off street',
 '7 spaces',
 'carport spaces',
 'electric door opener, slab parking spaces',
 'garage attached',

In [26]:
def get_spaces(x):        
    xsplit = x.split()
    
    if (len(xsplit) == 1) and (xsplit[0] == 'none'):
        return 0
    
    spaces_list = list(filter(lambda s: s.isnumeric(), x))
    
    if len(spaces_list) != 0:
        return spaces_list[0]
    
    return 1


hf_df['parking'].fillna('none', inplace=True)

better_data['p. spaces'] = hf_df['parking'].apply(get_spaces).astype(int)

get_vc(better_data['p. spaces'])

p. spaces
0    175425
1    139372
2     33480
3      5613
4      3665
6       966
5       836
8       544
7       197
9        94
Name: count, dtype: int64

In [27]:
parkings = ['garage', 'attached', 'detached', 
            'parking', 'carport', 'off street', 
            'on street', 'door opener', 'driveway', 
            'slab', 'assigned', 'open', 'none']

better_data['p. with spaces'] = (hf_df['parking']
                                 .apply(lambda x: 1 if (('space' in x) or 
                                                         (x[0].isnumeric())) else 0))

better_data = get_dummy_features('parking', labels=parkings, data=hf_df)


better_data.drop('p. with spaces', axis=1, inplace=True)

get_sums(better_data, 'parking')

parking garage          99320
parking attached        82656
parking detached        17445
parking parking          6119
parking carport         13812
parking off street       9131
parking on street        3381
parking door opener      2899
parking driveway         3308
parking slab              979
parking assigned         1026
parking open             3324
parking none           174348
parking other           61871
dtype: int64

### *Площадь участка*

In [28]:
get_populars(hf_df['lotsize'])

[nan,
 '—',
 '-- sqft lot',
 '0.26 acres',
 '0.25 acres',
 '0.28 acres',
 '0.27 acres',
 '0.29 acres',
 '0.34 acres',
 '0.31 acres',
 '0.32 acres',
 '6,098 sqft',
 '0.3 acres',
 '0.33 acres',
 '7,405 sqft',
 '0.35 acres',
 '6,534 sqft',
 '4,356 sqft',
 '10,000 sqft',
 '0.46 acres',
 '0.36 acres',
 '5,227 sqft',
 '1 acre',
 '0.37 acres',
 '5,000 sqft',
 '8,712 sqft',
 '3,920 sqft',
 '8,276 sqft',
 '0.38 acres',
 '0.39 acres',
 '0.41 acres',
 '0.5 acres',
 '10019',
 '10,019 sqft',
 '10000',
 '0.42 acres',
 '4,792 sqft',
 '0.43 acres',
 '0.44 acres',
 '0.4 acres',
 '9,583 sqft',
 '4,791 sqft',
 '10,454 sqft',
 '2,500 sqft',
 '0.45 acres',
 '7,840 sqft',
 '3,049 sqft',
 '6,969 sqft',
 '0.48 acres',
 '0.47 acres',
 '7,500 sqft',
 '6,000 sqft',
 '0.30 acres',
 '0.52 acres',
 '2,000 sqft',
 '0.51 acres',
 '6,970 sqft',
 '5,662 sqft',
 '871 sqft',
 '10018 sqft',
 '7405',
 '3,485 sqft',
 '4,000 sqft',
 '0.49 acres',
 '7,841 sqft',
 '0.53 acres',
 '5,663 sqft',
 '10890',
 '7,501 sqft',
 '0.55 ac

In [29]:
hf_df['lotsize'] = (hf_df['lotsize']
                    .fillna('--')
                    .apply(lambda x: 'none' if (('--' in x) or 
                                                ('—' in x)) else x))


def get_lotsize(x):
    xsplit = x.replace(',', '').split()
    
    if xsplit[0] == 'none':
        return -1
    
    if ((('sqft' in xsplit) or 
         ('sq.' in xsplit) or 
         (len(xsplit) == 1) and (xsplit[0] != 'none'))):
        return float(xsplit[0])
    
    if 'acres' in xsplit:
        acres = float(xsplit[0])
        return acres * 43560
    
better_data['lotsize'] = (hf_df['lotsize']
                          .apply(get_lotsize)
                          .astype(float)
                          .apply(lambda x: x / 1000 if x > 0 else x))


get_vc(better_data['lotsize'])

lotsize
-1.0000     85636
 10.8900     3437
 11.3256     3305
 12.1968     2768
 6.0980      2718
 7.4050      2696
 11.7612     2623
 12.6324     2485
 13.0680     2455
 10.0000     2320
 6.5340      2225
 5.0000      2059
 14.8104     2045
 5.2270      2010
 8.7120      2000
 4.3560      1977
 8.2760      1940
 13.5036     1809
 10.0190     1780
 13.9392     1639
Name: count, dtype: int64

### *Цена за квадратный фут*

Не нужен

### Почтовый индекс

In [30]:
data['zipcode'][:50]

0     28387
1     99216
2     90049
3     75205
4     32908
5     19145
6     34759
7     50401
8     77080
9     11354
10    77068
11    33028
12    97401
13    77084
14    11219
15    33311
16    10027
17    77375
18    33304
19    93552
20    97702
21    33139
22    77386
23    76542
24    20009
25    33183
26    77018
27    33328
28    11238
29    90016
30    33614
31    34952
32    33160
33    32210
34    28202
35    93705
36    33133
37    33610
38    75214
39    92127
40    33332
41    89108
42    91342
43    75709
44    76179
45    85310
46    32164
47    11357
48    33137
49    97221
Name: zipcode, dtype: object

In [31]:
data['zipcode'] = (data['zipcode']
                   .apply(lambda x: '00000' if ((x == '--') or 
                                                (len(x) != 5)) else x))

for i in range(5):
    better_data['zipcode '+str(i)] = data['zipcode'].apply(lambda x: int(x[i]))
    
better_data[['zipcode 0', 
             'zipcode 1', 
             'zipcode 2', 
             'zipcode 3', 
             'zipcode 4']]

,zipcode 0,zipcode 1,zipcode 2,zipcode 3,zipcode 4
0,2,8,3,8,7
1,9,9,2,1,6
2,9,0,0,4,9
3,7,5,2,0,5
4,3,2,9,0,8
...,...,...,...,...,...
360187,2,0,0,0,1
360188,3,3,1,8,0
360189,6,0,6,5,7
360190,1,1,4,3,4


### Школы

In [32]:
data['schools']

0         [{'rating': ['4', '4', '7', 'nr', '4', '7', 'n...
1         [{'rating': ['4/10', 'none/10', '4/10'], 'data...
2         [{'rating': ['8/10', '4/10', '8/10'], 'data': ...
3         [{'rating': ['9/10', '9/10', '10/10', '9/10'],...
4         [{'rating': ['4/10', '5/10', '5/10'], 'data': ...
                                ...                        
360187    [{'rating': ['3/10', '3/10'], 'data': {'distan...
360188    [{'rating': ['10/10', '5/10'], 'data': {'dista...
360189    [{'rating': ['1/10', '5/10', '7/10'], 'data': ...
360190    [{'rating': ['5/10', '4/10'], 'data': {'distan...
360191    [{'rating': ['5/10', '4/10', '3/10'], 'data': ...
Name: schools, Length: 360192, dtype: object

In [33]:
schools = data['schools'].apply(lambda x: x.replace('none, ', "'none', "))


schools_list = []

for school in schools:
    schools_le = ast.literal_eval(school)[0]
    schools_list.append(schools_le)
    
for schooldict in schools_list:
    schooldict['distance'] = schooldict['data']['distance']
    schooldict['grades'] = schooldict['data']['grades']
    
    del schooldict['data']

for schooldict in schools_list:
    for k, v in schooldict.items():
        if len(v) == 0:
            v.append('none')
        for elem in v:
            if elem == '':
                elem = 'none'


schools_df = pd.DataFrame(schools_list)

schools_df 

,rating,name,distance,grades
0,"[4, 4, 7, nr, 4, 7, nr, nr]","[southern pines elementary school, southern mi...","[2.7 mi, 3.6 mi, 5.1 mi, 4.0 mi, 10.5 mi, 12.6...","[3–5, 6–8, 9–12, pk–2, 6–8, 9–12, pk–5, k–12]"
1,"[4/10, none/10, 4/10]","[east valley high school&extension, eastvalley...","[1.65mi, 1.32mi, 1.01mi]","[9-12, 3-8, pk-8]"
2,"[8/10, 4/10, 8/10]","[paul revere middle school, brentwood science ...","[1.19mi, 2.06mi, 2.63mi]","[6-8, k-5, 9-12]"
3,"[9/10, 9/10, 10/10, 9/10]","[mcculloch intermediate school, bradfield elem...","[1.05mi, 0.1mi, 1.05mi, 0.81mi]","[5-6, pk-4, 7-8, 9-12]"
4,"[4/10, 5/10, 5/10]","[southwest middle school, bayside high school,...","[5.96mi, 3.25mi, 3.03mi]","[7-8, 9-12, pk-6]"
...,...,...,...,...
360187,"[3/10, 3/10]","[garrison elementary school, cardozo education...","[0.4 mi, 0.1 mi]","[pk-5, 6-12]"
360188,"[10/10, 5/10]","[air base elementary school, dr michael m. kro...","[32.1 mi, 1.1 mi]","[pk-8, 9-12]"
360189,"[1/10, 5/10, 7/10]","[hope college prep high school, lake view high...","[10.61mi, 1.42mi, 0.4mi]","[9-12, 9-12, pk-8]"
360190,"[5/10, 4/10]","[ps 48 william wordsworth, jhs 8 richard s gro...","[0.48mi, 0.73mi]","[pk-5, 6-8]"


### *Рейтинг школы*

In [34]:
schools_df['rating']

0         [4, 4, 7, nr, 4, 7, nr, nr]
1               [4/10, none/10, 4/10]
2                  [8/10, 4/10, 8/10]
3           [9/10, 9/10, 10/10, 9/10]
4                  [4/10, 5/10, 5/10]
                     ...             
360187                   [3/10, 3/10]
360188                  [10/10, 5/10]
360189             [1/10, 5/10, 7/10]
360190                   [5/10, 4/10]
360191             [5/10, 4/10, 3/10]
Name: rating, Length: 360192, dtype: object

In [35]:
ratings = (schools_df['rating']
           .apply(lambda x: ['-1'] if x[0] == '' else x)
           .apply(lambda x: [int(xs
                                 .replace('/10', '')
                                 .replace('-', '-1')
                                 .replace('none', '-1')
                                 .replace('nr', '-1')
                                 .replace('na', '-1')) 
                             for xs in x]))


better_data['school rating sum'] = ratings.apply(lambda x: np.sum(x))
better_data['school rating mean'] = ratings.apply(lambda x: np.mean(x))


better_data[['school rating sum', 'school rating mean']]

,school rating sum,school rating mean
0,23,2.875000
1,7,2.333333
2,20,6.666667
3,37,9.250000
4,14,4.666667
...,...,...
360187,6,3.000000
360188,15,7.500000
360189,13,4.333333
360190,9,4.500000


In [36]:
better_data['school rating sum'] = ratings.apply(lambda x: np.sum(x))
better_data['school rating mean'] = ratings.apply(lambda x: np.mean(x))


better_data[['school rating sum', 'school rating mean']]

,school rating sum,school rating mean
0,23,2.875000
1,7,2.333333
2,20,6.666667
3,37,9.250000
4,14,4.666667
...,...,...
360187,6,3.000000
360188,15,7.500000
360189,13,4.333333
360190,9,4.500000


### *Расстояние до школы*

In [37]:
schools_df['distance']

0         [2.7 mi, 3.6 mi, 5.1 mi, 4.0 mi, 10.5 mi, 12.6...
1                                  [1.65mi, 1.32mi, 1.01mi]
2                                  [1.19mi, 2.06mi, 2.63mi]
3                           [1.05mi, 0.1mi, 1.05mi, 0.81mi]
4                                  [5.96mi, 3.25mi, 3.03mi]
                                ...                        
360187                                     [0.4 mi, 0.1 mi]
360188                                    [32.1 mi, 1.1 mi]
360189                             [10.61mi, 1.42mi, 0.4mi]
360190                                     [0.48mi, 0.73mi]
360191                             [0.3 mi, 1.1 mi, 4.1 mi]
Name: distance, Length: 360192, dtype: object

In [38]:
distance = schools_df['distance'].apply(lambda x: [float(xs
                                                         .replace('none', '-1')
                                                         .replace(' ', '')
                                                         .replace('mi', '')) 
                                                   for xs in x])
     
better_data['school distance nearest'] = distance.apply(lambda x: np.min(x))
better_data['school distance mean'] = distance.apply(lambda x: np.mean(x))

better_data[['school distance nearest', 'school distance mean']]

,school distance nearest,school distance mean
0,2.70,5.537500
1,1.01,1.326667
2,1.19,1.960000
3,0.10,0.752500
4,3.03,4.080000
...,...,...
360187,0.10,0.250000
360188,1.10,16.600000
360189,0.40,4.143333
360190,0.48,0.605000


In [39]:
best = []
nearest = []

for i in data.index:
    rd_df = pd.DataFrame({'r': ratings[i], 'd': distance[i]})
    
    rat_n = rd_df.sort_values('d', ascending=False).iloc[0, 0]
    best.append(rat_n)
    
    dist_b = rd_df.sort_values('r', ascending=False).iloc[0, 1]
    nearest.append(dist_b)


better_data['school rating nearest'] = best
better_data['school distance to best'] = nearest

better_data[['school rating nearest', 'school distance to best']]

,school rating nearest,school distance to best
0,7,5.10
1,4,1.65
2,8,1.19
3,9,1.05
4,4,3.25
...,...,...
360187,3,0.40
360188,10,32.10
360189,1,0.40
360190,4,0.48


### *Уровни образования*

In [40]:
schools_df['grades']

0         [3–5, 6–8, 9–12, pk–2, 6–8, 9–12, pk–5, k–12]
1                                     [9-12, 3-8, pk-8]
2                                      [6-8, k-5, 9-12]
3                                [5-6, pk-4, 7-8, 9-12]
4                                     [7-8, 9-12, pk-6]
                              ...                      
360187                                     [pk-5, 6-12]
360188                                     [pk-8, 9-12]
360189                               [9-12, 9-12, pk-8]
360190                                      [pk-5, 6-8]
360191                                [pk-5, 6-8, 9-12]
Name: grades, Length: 360192, dtype: object

In [41]:
grades = schools_df['grades'].apply(lambda x: [(xs
                                                .replace(' to ', '-')
                                                .replace('–', '-')
                                                .replace('na', 'none') 
                                                .replace('n/a', 'none')) 
                                               for xs in x])

grades_set = set()

for grs in grades:
    for g in grs:
        grades_set.add(g)
        
grades_set

{'1-1',
 '1-10',
 '1-11',
 '1-12',
 '1-2',
 '1-3',
 '1-4',
 '1-5',
 '1-6',
 '1-7',
 '1-8',
 '1-9',
 '10-10',
 '10-11',
 '10-12',
 '11-11',
 '11-12',
 '12',
 '12-12',
 '2-11',
 '2-12',
 '2-2',
 '2-3',
 '2-4',
 '2-5',
 '2-6',
 '2-7',
 '2-8',
 '2-9',
 '3',
 '3-10',
 '3-11',
 '3-12',
 '3-4',
 '3-5',
 '3-5, 10',
 '3-6',
 '3-7',
 '3-8',
 '4-10',
 '4-11',
 '4-12',
 '4-4',
 '4-5',
 '4-6',
 '4-7',
 '4-8',
 '4-9',
 '5',
 '5-10',
 '5-11',
 '5-12',
 '5-5',
 '5-6',
 '5-7',
 '5-8',
 '5-9',
 '6',
 '6-10',
 '6-11',
 '6-12',
 '6-6',
 '6-7',
 '6-8',
 '6-8, 10',
 '6-9',
 '7',
 '7-10',
 '7-11',
 '7-12',
 '7-7',
 '7-8',
 '7-9',
 '8',
 '8-10',
 '8-11',
 '8-12',
 '8-8',
 '8-9',
 '9',
 '9-10',
 '9-11',
 '9-12',
 '9-9',
 'k',
 'k-1',
 'k-1, 3-5',
 'k-10',
 'k-11',
 'k-12',
 'k-2',
 'k-3',
 'k-4',
 'k-4, 6-12',
 'k-4, 6-8',
 'k-5',
 'k-6',
 'k-6, 8',
 'k-7',
 'k-8',
 'k-8, 10',
 'k-9',
 'k-k',
 'none',
 'pk, 1, 5',
 'pk, 1-4',
 'pk, 1-5',
 'pk, 1-6',
 'pk, 2',
 'pk, 3-5',
 'pk, 5-8',
 'pk, 6-8',
 'pk, 9-12',
 '

In [42]:
g_types = ['k', 'pk', 'preschool']

for t in g_types:
    better_data['schools with '+t] = (grades
                                      .apply(lambda x: sum([1 if t in xs else 0 
                                                            for xs in x]))
                                      .astype('uint8'))

get_sums(better_data, 'schools with')

schools with k            630684
schools with pk           440374
schools with preschool     65257
dtype: int64

In [43]:
def get_grades(x): 
    xnew = []
    
    for item in x:
        if ',' in item:
            item_splitted = item.split(', ')
            
            for splitted in item_splitted:
                xnew.append(splitted)
        
        else:
            xnew.append(item)
            
    xnew2 = []
    
    for item in xnew:
        if '-' in item:
            item_splitted = item.split('-')
            xrange = range(int(item_splitted[0]), 
                           int(item_splitted[-1])+1)
            
            for g in xrange:
                xnew2.append(g)
        
        else:
            xnew2.append(int(item))
            
    return len(set(xnew2))


better_data['school grades'] = (grades
                                .apply(lambda x: [(xs
                                                   .replace('preschool', '0')
                                                   .replace('pk', '0')
                                                   .replace('k', '0')
                                                   .replace('none', '0'))
                                                   for xs in x])
                                .apply(get_grades))

better_data['school grades']

0         13
1         13
2         13
3         13
4         13
          ..
360187    13
360188    13
360189    13
360190     9
360191    13
Name: school grades, Length: 360192, dtype: int64

### Цена недвижимости (целевой признак) 

In [44]:
data['target'][:50]

0       $418,000
1       $310,000
2     $2,895,000
3     $2,395,000
4         $5,000
5       $209,000
6        181,500
7       $244,900
8       $311,995
9       $669,000
10       260,000
11      $525,000
12      $499,900
13      $168,800
14     1,650,000
15       335,000
16     2,650,000
17      $365,000
18      $626,000
19      $375,000
20    $3,500,000
21       579,000
22      $499,007
23      $182,000
24    $3,749,000
25       799,000
26      $499,900
27      $559,000
28      $830,000
29    $1,195,000
30      $262,000
31      $204,800
32      $179,000
33      $105,000
34      $260,000
35      $284,900
36      $495,000
37      $117,900
38      $385,000
39    $1,100,000
40      $620,000
41      $125,000
42      $499,000
43    $1,429,000
44     $233,990+
45      $275,000
46       $27,000
47      $598,000
48    $1,780,000
49      $490,000
Name: target, dtype: object

In [45]:
better_data['target per month'] = (data['target']
                                   .apply(lambda x: 1 if '/mo' in x else 0)
                                   .astype('uint8'))

better_data['target plus'] = (data['target']
                              .apply(lambda x: 1 if '+' in x else 0)
                              .astype('uint8'))


better_data[['target per month', 'target plus']].sum()

target per month     398
target plus         7263
dtype: int64

In [46]:
def get_target(x):
    t_dict = {'$': ',', 
              ',': '/mo', 
              '/mo': '+', 
              '+': ''}
    
    for k, v in t_dict.items():
        x = x.replace(k, v)

    if x == 'unknown':
        return 0
        
    if '-' in x:
        xsplit = list(map(int, x.split(' - ')))
        x = np.mean(xsplit)
        
    return int(x) / 1000


better_data['target'] = data['target'].apply(get_target)

better_data['target']

0          418.000
1          310.000
2         2895.000
3         2395.000
4            5.000
            ...   
360187     799.000
360188    1249.000
360189     674.999
360190     528.000
360191     204.900
Name: target, Length: 360192, dtype: float64

## Загрузка обработанных признаков в отдельный датасет

In [47]:
better_data.to_csv('data/data_edited.csv', index=False)
del data

better_data

,status for sale,status coming soon,status new,status foreclosure,status pre-foreclosure,status pending,status auction,status active,status show,status for rent,...,school distance mean,school rating nearest,school distance to best,schools with k,schools with pk,schools with preschool,school grades,target per month,target plus,target
0,0,0,0,0,0,0,0,1,0,0,...,5.537500,7,5.10,3,2,0,13,0,0,418.000
1,1,0,0,0,0,0,0,0,0,0,...,1.326667,4,1.65,1,1,0,13,0,0,310.000
2,1,0,0,0,0,0,0,0,0,0,...,1.960000,8,1.19,1,0,0,13,0,0,2895.000
3,1,0,0,0,0,0,0,0,0,0,...,0.752500,9,1.05,1,1,0,13,0,0,2395.000
4,1,0,0,0,0,0,0,0,0,0,...,4.080000,4,3.25,1,1,0,13,0,0,5.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360187,1,0,0,0,0,0,0,0,0,0,...,0.250000,3,0.40,1,1,0,13,0,0,799.000
360188,0,0,0,0,0,0,0,0,0,0,...,16.600000,10,32.10,1,1,0,13,0,0,1249.000
360189,1,0,0,0,0,0,0,0,0,0,...,4.143333,1,0.40,1,1,0,13,0,0,674.999
360190,1,0,0,0,0,0,0,0,0,0,...,0.605000,4,0.48,1,1,0,9,0,0,528.000


*Смотрите продлжение работы в файле `modeling_nb.ipynp`*